## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [3]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin6_0 = pd.read_csv("data/bin+core_answer/"+'bin+core_answer_0.csv', index_col=0)
print(bin6_0.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin6_0.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
bin6_0.head()

(30092946, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [4]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin6_0['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin6_0['label'].value_counts())

256
0    30042688
1       50258
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [5]:
idx_bin = bin6_0[bin6_0['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 32 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin6_0), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin6_0', len(ls_idx_bin))

# loc 로 수정필요
bin6_0_Ngram = bin6_0.loc[ls_idx_bin,:].copy()

1608256
나머지 0
최종 길이 1608256
bin6_0 1608256


## (4) false data 만들기

In [6]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin6_0_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin6_0.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin6_0)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin6_0[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

50258.0
0 7667433
1000 20296320
2000 9302448
3000 14095508
4000 11150226
5000 8103839
6000 14168904
7000 25010267
8000 28749788
9000 7300523
10000 6771769
11000 18031361
12000 28367227
13000 24219159
14000 307353
15000 10302573
16000 440086
17000 23209342
18000 26781539
19000 11849275
20000 16811976
21000 13414996
22000 10148714
23000 12925773
24000 16272318
25000 28877847
26000 26838729
27000 5004694
28000 27927497
29000 23504863
30000 54190
31000 26436546
32000 1704470
33000 6678168
34000 8005989
35000 24822182
36000 23819021
37000 356838
38000 27954823
39000 16635900
40000 9188353
41000 18199930
42000 27571673
43000 10509981
44000 6419364
45000 14206527
46000 19710272
47000 22750668
48000 25900287
49000 2119122
50000 2557090
완료
50258


In [7]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin6_0['label'].value_counts()[1])

50258
50258


## (5) False Data + True Data 합치기

In [8]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin6_0_Ngram])
final.shape

(3216512, 2)

## (6) one hot encoding

In [9]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc6_0_onehot_Ngram = pd.get_dummies(final['bin'])
bc6_0_onehot_Ngram = pd.concat([final['label'], bc6_0_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc6_0_onehot_Ngram.shape)

원핫인코딩완료
(3216512, 257)


In [10]:
# 훈련 데이터, 훈련 라벨
x_bc6_0 = bc6_0_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc6_0 = bc6_0_onehot_Ngram['label'].to_numpy()
print(x_bc6_0.shape, x_bc6_0.shape)

x_bc6_0 = x_bc6_0.reshape(-1, right_idx, x_bc6_0.shape[1])
y_bc6_0 = y_bc6_0.reshape(-1, right_idx, 1)

print(x_bc6_0.shape, y_bc6_0.shape)

(3216512, 256) (3216512, 256)
(100516, 32, 256) (100516, 32, 1)


In [11]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc6_0.shape[0])

x_bc6_0 = x_bc6_0[p]
y_bc6_0 = y_bc6_0[p]

print(x_bc6_0.shape, y_bc6_0.shape)

(100516, 32, 256) (100516, 32, 1)


## (7) 모델

In [12]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(48, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [13]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc6_0, y_bc6_0):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc6_0[train],
               y_bc6_0[train],
               epochs = 10,
               batch_size = 128,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc6_0[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc6_0[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90464 samples
Epoch 1/10
90464/90464 [==============================] - 19s 207us/sample - loss: 0.0192 - accuracy: 0.9956
Epoch 2/10
90464/90464 [==============================] - 6s 64us/sample - loss: 3.8933e-04 - accuracy: 0.9999
Epoch 3/10
90464/90464 [==============================] - 6s 66us/sample - loss: 2.6348e-04 - accuracy: 0.9999
Epoc

accuracy_score 0.9999564763231198
recall_score 0.9994548428130111
precision_score 0.998003992015968
f1_score 0.9987288905029961
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90464 samples
Epoch 1/10
90464/90464 [==============================] - 13s 140us/sample - loss: 1.2462e-04 - accuracy: 1.0000
Epoch 2/10
90464/90464 [==============================] - 6s 63us/sample - loss: 1.1986e-04 

accuracy_score 0.9999595851571826
recall_score 0.9989266547406083
precision_score 0.9987479878375961
f1_score 0.9988373132993471
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90464 samples
Epoch 1/10
90464/90464 [==============================] - 12s 136us/sample - loss: 8.3696e-05 - accuracy: 1.0000
Epoch 2/10
90464/90464 [==============================] - 6s 65us/sample - loss: 7.2365e-05

accuracy_score 0.999975129327497
recall_score 0.9990982867448152
precision_score 0.9994587768356485
f1_score 0.9992784992784992
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90464 samples
Epoch 1/10
90464/90464 [==============================] - 13s 139us/sample - loss: 5.8040e-05 - accuracy: 1.0000
Epoch 2/10
90464/90464 [==============================] - 6s 67us/sample - loss: 4.6756e-05 

accuracy_score 0.9999844558296856
recall_score 0.9992932862190813
precision_score 0.9998232278592893
f1_score 0.9995581867986215
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90464 samples
Epoch 1/10
90464/90464 [==============================] - 13s 143us/sample - loss: 4.0694e-05 - accuracy: 1.0000
Epoch 2/10
90464/90464 [==============================] - 6s 71us/sample - loss: 3.4345e-05

...

accuracy_score 0.9999968911659372
recall_score 0.999822883457315
precision_score 1.0
f1_score 0.9999114338853955
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90465 samples
Epoch 1/10
90465/90465 [==============================] - 17s 188us/sample - loss: 1.3129e-05 - accuracy: 1.0000
Epoch 2/10
90465/90465 [==============================] - 6s 70us/sample - loss: 1.3572e-05 - accuracy: 1.0

accuracy_score 0.9999968908566311
recall_score 1.0
precision_score 0.9998167827042873
f1_score 0.9999083829592305
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90465 samples
Epoch 1/10
90465/90465 [==============================] - 13s 145us/sample - loss: 6.4101e-06 - accuracy: 1.0000
Epoch 2/10
90465/90465 [==============================] - 6s 70us/sample - loss: 1.2201e-05 - accuracy: 1.

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90465 samples
Epoch 1/10
90465/90465 [==============================] - 13s 148us/sample - loss: 3.4253e-06 - accuracy: 1.0000
Epoch 2/10
90465/90465 [==============================] - 6s 66us/sample - loss: 1.1003e-06 - accuracy: 1.0000
Epoch 3/10
90465/90465 [================

accuracy_score 0.9999968908566311
recall_score 1.0
precision_score 0.9998215560314061
f1_score 0.9999107700544303
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90465 samples
Epoch 1/10
90465/90465 [==============================] - 18s 194us/sample - loss: 4.9580e-06 - accuracy: 1.0000
Epoch 2/10
90465/90465 [==============================] - 6s 63us/sample - loss: 2.2347e-06 - accuracy: 1.

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0

K-fold cross validation Accuracy: [0.9999564763231198, 0.9999595851571826, 0.999975129327497, 0.9999844558296856, 0.9999844558296856, 0.9999968911659372, 0.9999968908566311, 1.0, 0.9999968908566311, 1.0]

K-fold cross validation Recall: [0.9994548428130111, 0.9989266547406083, 0.9990982867448152, 0.9992932862190813, 1.0, 0.999822883457315, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Precision: [0.998003992015968, 0.9987479878375961, 0.9994587768356485, 0.9998232278592893, 0.999091899745732, 1.0, 0.9998167827042873, 1.0, 0.9998215560314061, 1.0]

K-fold cross validation F1-Score: [0.9987288905029961, 0.9988373132993471, 0.9992784992784992, 0.9995581867986215, 0.9995457436176978, 0.9999114338853955, 0.9999083829592305, 1.0, 0.9999107700544303, 1.0]


## (9) 평가지표

In [14]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9999850775346371
10-Fold Cross_validation. Recall : 0.9996595953974831
10-Fold Cross_validation. Precision : 0.9994764223029927
10-Fold Cross_validation. F1-Score : 0.9995679220396217


In [15]:
model1.save('gcc6_bin_core_s32_h48_o0.h5')
print('save 완료')

save 완료
